/home/mohre/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import pandas as pd 
import os
DATA_PATH = "./data/raw/ph.csv"
df = pd.read_csv(DATA_PATH, index_col=None)
df.iloc[:10]['seq']

0    MAVEPLRVLVTGAAGQIGYALVPMIARGIMLGANQPVILHMLDIPP...
1    MKVAVLGAAGGIGQALALLLKTQLPAGSELSLYDIAPVTPGVAVDL...
2    MFATRQAFNLFQKRAFSASARQSSKVAILGAAGGIGQPLSLLMKLN...
3    MFATRQAFNLFQKRAFSASARQSSKVAILGAAGGIGQPLSLLMKLN...
4    MKLGFVGAGRVGSTSAFTCLLNLDVDEIALVDIAEDLAVGEAMDLA...
5    MPGPLRVLITGAAGQIAYNLSNMVANGNLFGKDQKIILHLLDIPEA...
6    MKTPVKIAVTGAAGQISYSLLFRLASGELLGPDQPMIFHLLETPQA...
7    MITIVGSGRVGTAAAAIMGIMRIDKKILLIDIVKGLPQGEALDLNH...
8    MKAPVRVAVTGAAGQIGYSLLFRIAAGEMLGKDQPVILQLLEIPQA...
9    MAPPQTIPRRGLFIGGAWREPCLGRRLPVVNPATEATIGDIPAGTA...
Name: seq, dtype: object

In [14]:
import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset

class ProteinSequenceDataset(Dataset):
    def __init__(self, sequence, targets, tokenizer, max_len):
        self.sequence = sequence
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, item):
        sequence = str(self.sequence[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            sequence,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
          'protein_sequence': sequence,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

In [17]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
# model = BertModel.from_pretrained("Rostlab/prot_bert")

In [18]:
train_dataset = ProteinSequenceDataset(sequence=df.iloc[:1000]['seq'],targets= df.iloc[:1000]['label'],tokenizer= tokenizer, max_len=512)

In [19]:
# Import modules that would be used later.
import os, random, pickle, itertools, copy
import learn2learn as l2l
import torchvision as tv
from PIL.Image import LANCZOS
from learn2learn.data.transforms import NWays, KShots, LoadData, RemapLabels, ConsecutiveLabels, FusedNWaysKShots
from learn2learn.vision.transforms import RandomClassRotation
from collections import defaultdict

In [20]:
ways = 2 
shots= 5
train_transforms = [
    FusedNWaysKShots(train_dataset, n=ways, k=2*shots),
    LoadData(train_dataset),
    RemapLabels(train_dataset),
    ConsecutiveLabels(train_dataset),
]
train_tasks = l2l.data.TaskDataset(train_dataset,
                                   task_transforms=train_transforms,
                                   num_tasks=20000)

AttributeError: 'ProteinSequenceDataset' object has no attribute 'labels'